In [4]:
import pickle

global eval_label
with open('./test_label.pkl', 'rb') as f:
    eval_label = pickle.load(f)

In [ ]:
predictions = {}
for k,v in eval_label.items():

    #Your inference code
    
    predictions[k] = []#list of extracted phrases from model

In [14]:
from nltk.stem.porter import *
from rouge import Rouge 
from tqdm.auto import tqdm
import numpy as np

rouge = Rouge()
stemmer = PorterStemmer()

def eval_score(reference, hypothesis):
    stem_h = [stemmer.stem(re.sub(r'[^\w\s]',' ',x)) for x in hypothesis]
    stem_h = list(set(stem_h))
    stem_r = [stemmer.stem(re.sub(r'[^\w\s]',' ',x)) for x in reference]  
    
    recall = []
    precision = []
    
    for h in stem_h:
        precision.append(max([rouge.get_scores(h, r)[0]['rouge-l']['p'] for r in stem_r]))
        
    for r in stem_r:
        if len(stem_h) > 0:
            recall.append(max([rouge.get_scores(h, r)[0]['rouge-l']['r'] for h in stem_h]))
        else:
            recall.append(0)
            
    recall = sum(recall)/len(reference)
    precision = sum(precision)/(len(hypothesis) if len(hypothesis) > 0 else 1e-8)
            
    return recall, precision

def run_eval(predictions):
    scores = []
    for k,v in tqdm(eval_label.items()):

        p = predictions.get(k)

        if p is not None:
            recall, precision = eval_score(v, p)
        else:
            recall, precision = 0, 0
        scores.append([recall, precision])
        
    scores = np.mean(np.array(scores), 0).tolist()
    return {f'Recall: {scores[0]}; Precision: {scores[1]}'}

In [15]:
run_eval(eval_label)

  0%|          | 0/7519 [00:00<?, ?it/s]

{'Recall: 1.0; Precision: 1.0'}

In [252]:
#Examples of how the metric works

for i, (k, v) in enumerate(eval_label.items()):
    p = predictions.get(k)
    R, P = eval_score(v, p)
    print(k)
    print(f'Ground Truth: {v}')
    print(f'Extraction: {p}')
    print(f'Recall: {R}; Precision: {P}')
    print('-----------------------------------------')
    
    if i == 30:
        break

3056778
Ground Truth: ['Accenture', 'EPAM', 'Endava', 'Globant']
Extraction: ['Accenture', 'EPAM', 'Endava', 'Globant']
Recall: 1.0; Precision: 1.0
-----------------------------------------
3057633
Ground Truth: ["Angie's List", 'FacilitySource', 'HomeAdvisor', 'Lawn Love', 'LawnStarter', 'Plowz & Mowz', 'SMS Assist', 'ServiceChannel', 'TaskEasy', 'Thumbtack']
Extraction: ["Angie's List", 'FacilitySource', 'HomeAdvisor', 'Lawn Love', 'LawnStarter', 'Plowz & Mowz', 'SMS Assist', 'ServiceChannel', 'TaskEasy', 'Thumbtack']
Recall: 1.0; Precision: 1.0
-----------------------------------------
3057683
Ground Truth: ['Claroty', 'Dragos', 'Nozomi Networks']
Extraction: ['Claroty', 'Dragos', 'Nozomi', 'Nozomi Networks']
Recall: 1.0; Precision: 1.0
-----------------------------------------
3058257
Ground Truth: ['Matterport']
Extraction: ['GeoCV', 'Immoviewer', 'Matterport', 'VTS']
Recall: 1.0; Precision: 0.25
-----------------------------------------
3059277
Ground Truth: ['Aera', 'Algomus', '